In [24]:
ENDPOINT_WHITELIST = "https://collaborative-training-auth.huggingface.co"

# Code stuff

In [25]:
import pprint
from getpass import getpass
from typing import List, Optional

import requests
from huggingface_hub import HfApi
from huggingface_hub.hf_api import HfApi, HfFolder
from pydantic import BaseModel, IPvAnyAddress, validator
from requests.exceptions import HTTPError

In [26]:
class HFTokenManagemment:
    def __init__(self):
        self._api = HfApi()

    def login(self):
        username = input("Experiment owner HF username: ")
        password = getpass()
        try:
            token = self._api.login(username, password)
        except HTTPError as e:
            # probably invalid credentials, display error message.
            print(e)
            print(e.response.text)
            exit(1)
        HfFolder.save_token(token)
        print("Login successful")
        print("Your token:", token, "\n")
        print("Your token has been saved to", HfFolder.path_token)

    def logout(self):
        token = HfFolder.get_token()
        if token is None:
            print("Not logged in")
            exit()
        HfFolder.delete_token()
        self._api.logout(token)
        print("Successfully logged out.")

In [27]:
# Define models to help request to API
class UserCreate(BaseModel):
    """
    username are required for registering a new user
    """

    username: str


class ExperimentFullCreatePublic(BaseModel):
    name: Optional[str]
    coordinator_ip: Optional[str]
    coordinator_port: Optional[int]
    collaborators: Optional[List[UserCreate]]

    @validator("coordinator_port")
    def validate_port(cls, port):
        if port is None:
            return port

        if int(port) > 2 ** 16:
            raise ValueError("port overflow")
        return port

In [28]:
def create_experiment(new_experiment: ExperimentFullCreatePublic):
    # Send request to server
    path = "{}/api/experiments/".format(ENDPOINT_WHITELIST)

    headers = {"Authorization": f"Bearer {HfFolder.get_token()}"}
    r = requests.post(
        path, headers=headers, json={"new_experiment": new_experiment.dict()}
    )
    r.raise_for_status()
    return r.json()

# Login to HF account

In [ ]:
# As a HF user, you ask a HF token session
token_management = HFTokenManagemment()
token_management.login()

# Create a new experiment

The next cell need to be updated with the values you want

In [29]:
# This is the content request for the new experiment to create

new_experiment = ExperimentFullCreatePublic(
    name="<Add Experiment name HERE>",  # Change ME
    #     coordinator_ip= <Add IP HERE>, # Change ME, Optionnal, str type
    #     coordinator_port= <Add Port HERE>, # Change ME, Optionnal, int type
)

In [30]:
response = create_experiment(new_experiment)

print("API response:")
pprint.pprint(response)
print(f"\nExperiment id: {response['id']}")

API response:
{'collaborators': None,
 'coordinator_ip': None,
 'coordinator_port': None,
 'created_at': '2021-05-11T19:21:11.373082+00:00',
 'id': 6,
 'name': '<Add Experiment name HERE>',
 'owner': 'Aline',
 'updated_at': '2021-05-11T19:21:11.373082+00:00'}

Experiment id: 6


# Logout 
Really import to run this last cell for security reasons

In [ ]:
# As a HF user, you terminate your session (your token is not longer valid)
token_management.logout()